<a href="https://colab.research.google.com/github/joseprotger/Coursera_Capstone/blob/main/Capstone_Project_Coursera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IBM Data Science Professional Certificate
## Capstone Project
This notebook will be used to develop the IBM Data Science Capstone Project on Coursera:

**Battle of the neighborhoods**

In [2]:
import pandas as pd
import numpy as np

In [1]:
print("Hello Capstone Project course!")

Hello Capstone Project course!
